In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [5]:
datasets = pd.read_csv('exports/pdokWebsiteDatasets.csv')

In [6]:
# init dict
serviceValues = []

# loop over datasets
for datasetTitelID in datasets.titel.tolist():
    serviceURL = datasets.loc[ datasets['titel'] == datasetTitelID ]['link'].item() # get link
    serviceURL = serviceURL.replace("introductie", "geo-services") # go to services tab
    
    #soup
    response = requests.get(serviceURL)
    soup = BeautifulSoup(response.content, 'html.parser')
    t = soup.findAll('table') # all tables


    # loop over tables
    for table in t:

        rows = table.findAll('tr')

        # loop over the 4 rows per table
        values = {} # init list for values

        for i in rows:
            if i.td.a != None: # in case the value is a link, grab the href
                values.update( {i.th.text.strip() : i.td.a['href']} )
                continue
            values.update( {i.th.text.strip() : i.td.text.strip()} )
            values.update({"Dataset Titel": datasetTitelID})

        # store the table values as a dict with id titel
        serviceValues.append(values)

In [9]:
serviceDF = pd.DataFrame.from_dict(serviceValues)
serviceDF.to_csv('exports/pdokWebsiteServices.csv', header=True, index=False)